In [ ]:
!pip install transformers torchaudio jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 33.9 MB/s eta 0:00:00


In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torchaudio
import torch
import jiwer

# Load the pre-trained model and processor
model_name = "facebook/wav2vec2-base-960h"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name).to("cuda")

# Set the LibriSpeech dataset configuration
librispeech_dataset = torchaudio.datasets.LIBRISPEECH("./", url="test-other", download=True)

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You sho

In [ ]:
import torchaudio
from torch.utils.data import Dataset
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import jiwer
import os

# Load the pre-trained model and processor
model_name = "facebook/wav2vec2-base-960h"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You sho

## Split by female and male

In [ ]:
import torchaudio
from torch.utils.data import Dataset
import os
import shutil

class GenderSubsetDataset(Dataset):
    def __init__(self, root, subset, utterances):
        self.root = root
        self.subset = subset
        self.utterances = utterances
        self.wav_files = [f for f in os.listdir(os.path.join(root, subset)) if f.endswith(".wav")]

    def __len__(self):
        return len(self.wav_files)

    def __getitem__(self, idx):
        wav_file = self.wav_files[idx]
        wav_path = os.path.join(self.root, self.subset, wav_file)
        waveform, sample_rate = torchaudio.load(wav_path)

        # Include the corresponding utterance
        index = int(wav_file.split(".")[0])
        utterance = self.utterances[index]

        return waveform, utterance, wav_file
# Load the LibriSpeech dataset configuration
# librispeech_dataset = torchaudio.datasets.LIBRISPEECH("./", url="test-clean", download=True)

# Load information from SPEAKERS.TXT
speakers_file_path = "LibriSpeech/SPEAKERS.TXT"  # Replace with the actual path
speaker_info = {}

with open(speakers_file_path, 'r') as file:
    for line in file:
        if line.startswith(';') or not line.strip():
            continue
        fields = line.strip().split('|')
        speaker_id = int(fields[0].strip())
        gender = fields[1].strip()
        speaker_info[speaker_id] = gender


In [ ]:
# Set paths for male and female subsets
male_subset_path = "librispeech_male"
female_subset_path = "librispeech_female"

# Create directories if they don't exist
os.makedirs(male_subset_path, exist_ok=True)
os.makedirs(female_subset_path, exist_ok=True)

utterances_female = {}
utterances_male = {}

# Process the LibriSpeech dataset
for i in range(len(librispeech_dataset)):
    waveform, sample_rate, utterance, speaker_id, _, _ = librispeech_dataset[i]

    # Determine gender based on information from SPEAKERS.TXT
    gender = speaker_info.get(speaker_id, "unknown")

    # Move the audio file to the appropriate subset directory
    if gender == "M":
        destination_path = os.path.join(male_subset_path, f"{i}.wav")
        utterances_male[i] = utterance
    elif gender == "F":
        destination_path = os.path.join(female_subset_path, f"{i}.wav")
        utterances_female[i] = utterance
    else:
        continue  # Skip if gender is unknown

    torchaudio.save(destination_path, waveform, sample_rate)

    print(f"Processed example {i + 1}: Gender - {gender}")

print(f"Finished processing. Male subset: {male_subset_path}, Female subset: {female_subset_path}")

Processed example 1: Gender - M
Processed example 2: Gender - M
Processed example 3: Gender - M
Processed example 4: Gender - M
Processed example 5: Gender - M
Processed example 6: Gender - M
Processed example 7: Gender - M
Processed example 8: Gender - M
Processed example 9: Gender - M
Processed example 10: Gender - M
Processed example 11: Gender - M
Processed example 12: Gender - M
Processed example 13: Gender - M
Processed example 14: Gender - M
Processed example 15: Gender - M
Processed example 16: Gender - M
Processed example 17: Gender - M
Processed example 18: Gender - M
Processed example 19: Gender - M
Processed example 20: Gender - M
Processed example 21: Gender - M
Processed example 22: Gender - M
Processed example 23: Gender - M
Processed example 24: Gender - M
Processed example 25: Gender - M
Processed example 26: Gender - M
Processed example 27: Gender - M
Processed example 28: Gender - M
Processed example 29: Gender - M
Processed example 30: Gender - M
Processed example 3

In [ ]:
# Create dataset instances
import IPython.display as ipd

male_dataset = GenderSubsetDataset(root=male_subset_path, subset="", utterances=utterances_male)
female_dataset = GenderSubsetDataset(root=female_subset_path, subset="", utterances=utterances_female)

# Example: Accessing a sample from the male dataset
sample_waveform, utterance, sample_filename = male_dataset[0]
print(f"Sample from the male dataset: {sample_filename}, utterance = {utterance}")
ipd.Audio(data=sample_waveform, autoplay=True, rate=16000)


Sample from the male dataset: 518.wav, utterance = I WENT AROUND AND CLUMB OVER THE BACK STILE BY THE ASH HOPPER AND STARTED FOR THE KITCHEN


In [ ]:
# Example: Accessing a sample from the female dataset
sample_waveform, utterance, sample_filename = female_dataset[0]
print(f"Sample from the female dataset: {sample_filename}, utterance = {utterance}")
ipd.Audio(data=sample_waveform, autoplay=True, rate=16000)


Sample from the female dataset: 725.wav, utterance = THREE YEARS AGO MADAME DE BETHUNE A JANSENIST TURNED ORTHODOX MERELY FROM HAVING SEEN MOTHER CRUCIFIXION AT PRAYER AH


In [ ]:
def evaluate(dataset):
    predictions = []
    ground_truths = []

    for i in range(len(dataset)):
        waveform, utterance, _ = dataset[i]

        # Preprocess the audio file
        input_values = processor(waveform.numpy(), return_tensors="pt").input_values

        # Perform inference
        with torch.no_grad():
            logits = model(input_values).logits

        # Decode the logits to obtain the transcription
        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = processor.batch_decode(predicted_ids)[0]

        predictions.append(transcription)
        ground_truths.append(utterance)

    return predictions, ground_truths

In [ ]:
# Evaluate WER for male subset
print("Evaluate WER for Male Subset:")
male_predictions, male_ground_truths = evaluate(male_dataset)
male_average_wer = jiwer.wer(male_predictions, male_ground_truths)
print(f"Average WER for Male Subset: {male_average_wer}\n")

# Evaluate WER for female subset
print("Evaluate WER for Female Subset:")
female_predictions, female_ground_truths = evaluate(female_dataset)
female_average_wer = jiwer.wer(female_predictions, female_ground_truths)
print(f"Average WER for Female Subset: {female_average_wer}")

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Evaluate WER for Male Subset:


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_ra

Average WER for Male Subset: 0.09581996055836975

Evaluate WER for Female Subset:


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_ra

Average WER for Female Subset: 0.07579018919225104


In [ ]:
# Evaluate WER for male subset
print("Evaluate WER for Male Subset:")
male_predictions, male_ground_truths = evaluate(male_dataset)
male_average_wer = jiwer.wer(male_predictions, male_ground_truths)
print(f"Average WER for Male Subset: {male_average_wer}\n")

# Evaluate WER for female subset
print("Evaluate WER for Female Subset:")
female_predictions, female_ground_truths = evaluate(female_dataset)
female_average_wer = jiwer.wer(female_predictions, female_ground_truths)
print(f"Average WER for Female Subset: {female_average_wer}")

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Evaluate WER for Male Subset:


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_ra

Average WER for Male Subset: 0.09581996055836975

Evaluate WER for Female Subset:


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_ra

Average WER for Female Subset: 0.07579018919225104
